<a href="https://colab.research.google.com/github/hogehogetakumi/TakumiTechNotes.github.io/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -qU langchain langchain-community pypdf langchain-google-genai docx2txt faiss-cpu

In [8]:
import os
import re
from pathlib import Path
from google.colab import userdata
from langchain.schema import Document, HumanMessage, AIMessage
from langchain.document_loaders import TextLoader, Docx2txtLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS


In [4]:
def preprocess(doc: Document) -> Document:
    text = doc.page_content
    # ヘッダー／フッターの簡易除去
    text = re.sub(r'第[一二三四五六七八九十]+章.*\n', '', text)
    text = re.sub(r'Copyright.*\n', '', text)
    text = re.sub(r'\n\s*\n+', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'([^\。\?\!])\n([^\n])', r'\1\2', text)
    meta = {
        'source': doc.metadata.get('source'),
        'page': doc.metadata.get('page')
    }
    return Document(page_content=text.strip(), metadata=meta)

In [5]:
def load_documents_from_folder(folder_path: str):
    docs = []
    for fname in os.listdir(folder_path):
        path = Path(folder_path) / fname
        if not path.is_file(): continue
        try:
            suffix = path.suffix.lower().lstrip('.')
            if suffix == "pdf":
                loader = PyPDFLoader(path)
            elif suffix in ("txt", "md"):
                loader = TextLoader(path, encoding="utf-8")
            elif suffix == "docx":
                loader = Docx2txtLoader(path)
            else:
                print(f"Skipping unsupported: {fname}")
                continue
            docs.extend(loader.load())
        except Exception as e:
            print(f"Error loading {fname}: {e}")
    return docs

In [10]:
def main():
    # ─── 1. ドキュメント読み込み＆前処理 ───
    raw_docs = load_documents_from_folder("data")
    clean_docs = [preprocess(d) for d in raw_docs]

    # ─── 2. チャンク分割 ───
    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(clean_docs)

    # ─── 3. 埋め込み＆ベクトルストア構築 ───
    api_key = userdata.get("GEMINI_API_KEY")
    embeddings = GoogleGenerativeAIEmbeddings(
        google_api_key=api_key,
        model="models/embedding-001"
    )
    vectorstore = FAISS.from_documents(chunks, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # ─── 4. LLM 初期化 ───
    llm = ChatGoogleGenerativeAI(
        google_api_key=api_key,
        model="gemini-2.5-flash",
        temperature=0
    )

    # 会話履歴は HumanMessage/AIMessage のリストで保持
    conversation_history = []

    # ─── 5. チャットループ ───
    while True:
        user_input = input("あなた: ")
        if user_input.lower() in ("", "exit", "quit"):
            print("会話を終了します。")
            break

        # 履歴にユーザーメッセージを追加
        conversation_history.append(HumanMessage(content=user_input))

        # ─── 5-1. リトリーバル ───
        docs = retriever.get_relevant_documents(user_input)
        # メタ情報付きでまとめてプロンプトに含める
        context = "\n\n".join(
            f"[{d.metadata['source']} p.{d.metadata['page']}]\n{d.page_content}"
            for d in docs
        )

        # ─── 5-2. LLM 呼び出し ───
        # ここでは、まず「ドキュメント参照指示」→ 会話履歴 → AI に渡します
        messages = []
        messages.append(HumanMessage(
            content=f"以下のドキュメントを参照して回答してください：\n{context}\n\n質問: {user_input}"
        ))
        # 過去の会話履歴も続けて渡す
        messages.extend(conversation_history[:-1])  # 最新のユーザーメッセージは既に先頭に含めたので除外

        # 実際の呼び出し
        ai_message: AIMessage = llm(messages)
        conversation_history.append(ai_message)

        # ─── 5-3. 応答表示 ───
        print("アシスタント:", ai_message.content)

if __name__ == "__main__":
    main()

あなた: exit
会話を終了します。
